In [16]:
!pip install streamlit

   ---------------------------------------- 0.0/9.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.1 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.1 MB 2.3 MB/s eta 0:00:04
   ---- ----------------------------------- 1.0/9.1 MB 3.4 MB/s eta 0:00:03
   ---------------- ----------------------- 3.7/9.1 MB 6.2 MB/s eta 0:00:01
   ---------------------- ----------------- 5.2/9.1 MB 6.6 MB/s eta 0:00:01
   ----------------------------- ---------- 6.8/9.1 MB 7.0 MB/s eta 0:00:01
   ------------------------------------- -- 8.7/9.1 MB 7.3 MB/s eta 0:00:01
   ---------------------------------------- 9.1/9.1 MB 7.2 MB/s  0:00:01
   ---------------------------------------- 0.0/795.4 kB ? eta -:--:--
   ---------------------------------------- 795.4/795.4 kB 9.8 MB/s  0:00:00
   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ---------- ----------------------------- 1.8/6.9 MB 9.4 MB/s eta 0:00:01
   --------------------- ---------------


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
# Installation des outils de visualisation pour le niveau professionnel
!pip install seaborn matplotlib


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import re
import pickle

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report


In [6]:
import os
import pandas as pd

# 1. Créer le dossier 'data' s'il n'existe pas
if not os.path.exists('data'):
    os.makedirs('data')

# 2. Créer des données d'exemple
donnees = {
    'message': [
        "Félicitations ! Vous avez gagné 1000€, cliquez ici.",
        "Salut, tu viens manger ce soir ?",
        "Urgent : votre compte est bloqué, connectez-vous vite.",
        "N'oublie pas d'acheter du pain en rentrant.",
        "Investissez 100€ et gagnez 10000€ en 2 jours !",
        "Bonjour, voici la facture pour votre abonnement."
    ],
    'label': ['arnaque', 'legitime', 'arnaque', 'legitime', 'arnaque', 'legitime']
}

# 3. Enregistrer le fichier dans le dossier data
df_nouveau = pd.DataFrame(donnees)
df_nouveau.to_csv("data/messages.csv", index=False)

print("Le fichier 'data/messages.csv' a été créé avec succès !")

Le fichier 'data/messages.csv' a été créé avec succès !


In [7]:
df = pd.read_csv("data/messages.csv")
df.head()

,message,label
0,"Félicitations ! Vous avez gagné 1000€, cliquez...",arnaque
1,"Salut, tu viens manger ce soir ?",legitime
2,"Urgent : votre compte est bloqué, connectez-vo...",arnaque
3,N'oublie pas d'acheter du pain en rentrant.,legitime
4,Investissez 100€ et gagnez 10000€ en 2 jours !,arnaque


In [32]:
import pandas as pd
import re
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# --- 1. FONCTION DE NETTOYAGE EXPERT ---
def clean_text_expert(text):
    # Passage en minuscules
    text = text.lower()
    # Remplacement des URLs par un tag spécifique (conserve l'indice de danger)
    text = re.sub(r'http\S+|www\S+', ' [url_détecté] ', text)
    # Remplacement des montants financiers (crucial pour le phishing local)
    text = re.sub(r'\d+', ' [montant_chiffré] ', text)
    # On garde les lettres et nos tags [ ]
    text = re.sub(r'[^a-zàâçéèêëîïôûùÿñæœ\[\]\s]', ' ', text)
    # Suppression des espaces multiples
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# --- 2. PRÉPARATION DES DONNÉES ---
# On utilise ton dictionnaire enrichi 'data_finale'
df = pd.DataFrame(data_finale)
df['message_clean'] = df['message'].apply(clean_text_expert)

# Séparation équilibrée (80% apprentissage, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    df['message_clean'], 
    df['label'], 
    test_size=0.20, 
    random_state=42,
    stratify=df['label'] # Important pour garder le même ratio arnaque/sain
)

# --- 3. VECTORISATION AVEC N-GRAMMES ---
# On analyse les mots seuls ET les groupes de 2 mots (ex: "compte bloqué")
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# --- 4. ENTRAÎNEMENT DU MODÈLE ---
model = MultinomialNB(alpha=0.1) # Alpha ajusté pour la précision sur petits volumes
model.fit(X_train_tfidf, y_train)

# --- 5. ÉVALUATION ET EXPORT ---
score = model.score(X_test_tfidf, y_test) * 100
print(f"✅ Performance du modèle : {score:.2f}%")

# Sauvegarde pour Streamlit
pickle.dump(model, open("model.pkl", "wb"))
pickle.dump(vectorizer, open("vectorizer.pkl", "wb"))
print("🚀 Fichiers model.pkl et vectorizer.pkl mis à jour !")

✅ Performance du modèle : 100.00%
🚀 Fichiers model.pkl et vectorizer.pkl mis à jour !


In [44]:
import pandas as pd
import re
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# 1. LISTE DES ARNAQUES
arnaques = [
    "Félicitations ! Vous avez gagné 1000000 d'euros. Cliquez ici pour réclamer.",
    "Urgent virement : votre compte est bloqué, connectez-vous vite.",
    "Votre colis est en attente, payez 2€ de frais de port ici.",
    "Alerte sécurité : tentative de connexion suspecte sur votre compte bancaire.",
    "Gagnez un iPhone 15 gratuitement en répondant à ce sondage.",
    "Urgent : Rappelez ce numéro surtaxé pour votre gain de 50000 F.",
    "Votre abonnement Netflix a expiré, mettez à jour vos coordonnées bancaires.",
    "Virement reçu de 150000 F. Cliquez pour valider le transfert.",
    "Besoin d'aide financière ? Prêt immédiat sans justificatif.",
    "Coucou maman, j'ai cassé mon téléphone, envoie-moi de l'argent sur ce numéro.",
    "Investissez 10000 F et gagnez 100000 F en 24h seulement !",
    "Frais de douane impayés pour votre colis international.",
    "Votre compte CPF va expirer, utilisez vos droits maintenant.",
    "Gagnez un bon d'achat de 500€ chez Amazon via ce lien.",
    "Urgent virement suspect détecté, confirmez votre identité immédiatement.",
    "Offre d'emploi à domicile : gagnez 3000€ par mois sans expérience.",
    "Félicitations, vous êtes le 1 000 000e visiteur, gagnez un cadeau.",
    "Votre dossier bancaire nécessite une mise à jour urgente.",
    "Erreur de facturation : nous vous devons 200€. Cliquez pour le remboursement.",
    "Achat de 50000 F confirmé. Si vous n'êtes pas l'auteur, cliquez ici.",
    "Un crédit a été effectué sur votre compte Mobile Money. Merci de rappeler le +228 90 90 90 90.",
    "Votre virement de 659,99 € pour le bénéficiaire Marc Dupont est validé. Pour annuler cette opération, contactez le service opposition au XXXXXX XXXX.",
    "Salut..!Comme vous le savez c'est votre international Emmanuel adebayor shéyi..." # Garde ton texte complet ici
]

# 2. LISTE DES MESSAGES LÉGITIMES
legitimes = [
    "Mixx x HABIBI : FUFU Power King à 97 750 FCFA au lieu de 115 000.",
    "Merci de me rappeler. Je n'ai pas suffisamment de credit.",
    "Félicitations pour avoir cumulé 512 points. Continue de consommer...",
    "Bénéficie de 50% de bonus val 1j sur tout achat de forfait appel...",
    "Apprendre, c'est gagner ! Avec EduQuiz, améliore ta culture...",
    "Pharmacies de garde, horaires d'avions, presse et plus...",
    "Bonjour, tu viens manger ce soir ?",
    "Réunion de projet demain à 10h dans la salle B.",
    "Le cours de Python est déplacé à lundi prochain.",
    "Est-ce que tu as fini ton rapport pour la soutenance ?",
    "Ton code de vérification pour ton compte est 4589.",
    "Je t'envoie le document PDF par mail.",
    "On se voit au café à 15h ?",
    "Bonjour",
    "Merci pour ton aide sur cet exercice.",
    "Tu as vu le match de foot hier soir ?",
    "Le rendez-vous chez le dentiste est confirmé pour mardi.",
    "Pense à charger ton ordinateur pour demain.",
    "C'est quoi le programme du week-end ?",
    "Je serai un peu en retard à la réunion.",
    "Peux-tu m'envoyer le numéro de téléphone de Jean ?",
    "J'ai bien reçu ton message, je te rappelle plus tard.",
    "Bonne chance pour ta présentation de demain !",
    "N'oublie pas de fermer la porte à clé.",
    "On se retrouve à l'entrée de l'université.",
    "Est-ce que tu peux m'aider avec mon code ?"
]

# Création du DataFrame (SANS ERREUR DE LONGUEUR)
df = pd.DataFrame({
    'message': arnaques + legitimes,
    'label': ['arnaque'] * len(arnaques) + ['legitime'] * len(legitimes)
})

# Nettoyage et Entraînement
def clean_text_pro(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+', ' [url_détecté] ', text)
    text = re.sub(r'\d+', ' [montant_chiffré] ', text)
    text = re.sub(r'[^a-zàâçéèêëîïôûùÿñæœ\[\]\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['message'] = df['message'].apply(clean_text_pro)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['message'])
model = MultinomialNB()
model.fit(X, df['label'])

# Sauvegarde
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

print(f"✅ Terminé : {len(df)} messages traités !")

✅ Terminé : 49 messages traités !
